In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w10p1/train.csv
/kaggle/input/2022-ai-w10p1/test.csv
/kaggle/input/2022-ai-w10p1/sample_submit.csv


In [88]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.init
import random
import cv2

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [90]:
#데이터 가져오기
import numpy as np 
import pandas as pd

train_data=pd.read_csv('/kaggle/input/2022-ai-w10p1/train.csv')
train_data

,Category,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
x_train=train_data.iloc[:,1:]
x_train

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
y_train=train_data.iloc[:,0]
y_train

0        5
1        0
2        3
3        3
4        6
        ..
59995    0
59996    3
59997    9
59998    4
59999    6
Name: Category, Length: 60000, dtype: int64

In [93]:
x_test = pd.read_csv("/kaggle/input/2022-ai-w10p1/test.csv")
x_test

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)

In [95]:
y_train=y_train.reshape(-1,1)
y_train

array([[5],
       [0],
       [3],
       ...,
       [9],
       [4],
       [6]])

In [96]:
#method="resnet"
method="vgg"
#method="alexnet"

In [97]:
import torchvision.models as models

if method == "resnet":
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
elif method == "vgg":
    model = models.vgg16(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
elif method == "alexnet":
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
else:
    raise ValueError("Check Method!")

In [98]:
model=model.to(device)

In [99]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 1
batch_size = 100

In [100]:
#데이터 로더 설정
class Custom_Loader(torch.utils.data.Dataset):
    def __init__(self, data, label=None, split='train', transform=None):
        self.label = label
        self.split = split
        self.transform = transform
        
        # csv 파일로부터 얻어진 데이터를 28, 28, 1로 shape 변환
        data_1ch = np.reshape(data, (-1, 28, 28, 1)) # (60000, 28, 28, 1) 
        # 1ch 의 MNIST이미지를 3ch 로 변환 
        data_3ch = data_1ch.repeat(3, axis=-1) # (60000, 28, 28, 3) 
        # data 인스턴스에 변환한 데이터 저장
        self.data = data_3ch.astype(np.uint8)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # idx번째 데이터를 data에 저장
        data = self.data[idx]  # shape (28, 28, 3)
        
        # 28x28 이미지를 224x224로 resize
        data = cv2.resize(data, dsize=(224, 224), interpolation=cv2.INTER_LINEAR) # shape (224, 224, 3)
        
        # Tensor 변환 및 Normalization 
        data = self.transform(data)
        
        # train loader 일 경우 data, label 반환
        if self.split == 'train':
            return data, torch.from_numpy(self.label[idx])
        else:
        # test 일 경우, data만 반환 
            return data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = Custom_Loader(x_train, label=y_train, split='train', transform=transform)
test_dataset = Custom_Loader(x_test, split='test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [101]:
#손실함수와 최적화 함수 선언
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [102]:
total_batch = len(train_loader)
print('Learning started. It takes some time.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.squeeze(-1).to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.064866811
Learning Finished!


In [103]:
# 학습된 모델로 test 데이터 예측
prediction=torch.empty(0).to(device)
with torch.no_grad():
    model.eval()
    
    for X in test_loader:
        X=X.to(device)
        predict=model(X)
        
        prediction=torch.cat((prediction,torch.argmax(predict,1)),dim=0)
        #1은 차원으로 각 행에서 가장 큰 값을 가진 인덱스를 반환한다.

In [110]:
prediction=prediction.type(torch.int64)

In [112]:
# csv 변환
prediction = prediction.cpu().numpy()
submit = pd.read_csv("/kaggle/input/2022-ai-w10p1/sample_submit.csv")
submit['Category'] = prediction
submit

,Id,Category
0,0,6
1,1,1
2,2,0
3,3,0
4,4,9
...,...,...
9995,9995,3
9996,9996,7
9997,9997,2
9998,9998,8


In [ ]:
submit.to_csv('submission.csv',index=False)